In [1]:
# import os
# os.environ['HF_HOME']= "/Users/bytedance/Desktop/void-diffusion"
# os.environ['http_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy


from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from transformer import Transformer

In [3]:
torch.backends.mps.is_built(),torch.backends.mps.is_available(),torch.cuda.is_available()

(True, True, False)

In [4]:

if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

# device = torch.device('cpu')

memory usage?
flops
reproduce experiments
ablation study
what else except attention/ architecture properties / mechanisms

In [5]:
from datasets import load_dataset
dataset = load_dataset("wmt14", 'de-en')

/Users/bytedance/Desktop/void-diffusion/env/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for wmt14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wmt14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2") # split + toInt
tokenizer.pad_token = tokenizer.eos_token
tokenizer.vocab_size

50257

In [7]:
src_vocab_size = tokenizer.vocab_size
tgt_vocab_size = tokenizer.vocab_size
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 512
dropout = 0.1
batch_size = 64

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)

# # Generate random sample data
# src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
# tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [8]:
# for batch in dataset['train'].iter(batch_size):
#     translation = batch['translation']
    
#     source = [sample['de'] for sample in translation]
#     target = [sample['en'] for sample in translation]
#     source_tokenized = tokenizer(source, padding='max_length',max_length=max_seq_length)
#     target_tokenized = tokenizer(target, padding='max_length',max_length=max_seq_length)
    
#     source_tensor = torch.tensor(source_tokenized['input_ids'], dtype=torch.long)
#     target_tensor = torch.tensor(target_tokenized['input_ids'], dtype=torch.long)
#     break


In [9]:
def tokenization(batch):
    translation = batch['translation']
    source = [sample['de'] for sample in translation]
    target = [sample['en'] for sample in translation]
    source_tokenized = tokenizer(source, padding='max_length',max_length=max_seq_length)
    target_tokenized = tokenizer(target, padding='max_length',max_length=max_seq_length)
    
    source_tensor = torch.tensor(source_tokenized['input_ids'], dtype=torch.long).to(device)
    target_tensor = torch.tensor(target_tokenized['input_ids'], dtype=torch.long).to(device)
    
    return source_tensor, target_tensor

In [10]:
src_data,tgt_data = tokenization(dataset['train'][:1])

In [11]:
for i in dataset['train'].iter(1):
    print(i)
    break

{'translation': [{'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}]}


In [17]:
import time

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    print(output.shape, tgt_data.shape,output.contiguous().view(-1, tgt_vocab_size).shape)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()} Time:{time.time()}")

mps:0 mps:0 mps:0
torch.Size([1, 511, 50257]) torch.Size([1, 512]) torch.Size([511, 50257])
Epoch: 1, Loss: 0.033663731068372726 Time:1711615569.145741
mps:0 mps:0 mps:0
torch.Size([1, 511, 50257]) torch.Size([1, 512]) torch.Size([511, 50257])


KeyboardInterrupt: 

In [ ]:
transformer.eval()

# Generate random sample validation data
val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")